In [ ]:
%load_ext autoreload
%autoreload 2

# Author

> Author Class and all methods to get details of each author form the publication

In [ ]:
#| default_exp author

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from Bio import Entrez
import sys
# from tinydb import TinyDB, Query, where
import pandas as pd
from datetime import datetime, timedelta, date
from collections import defaultdict, Counter
import  pickle
from fastcore.all import *
from dotenv import load_dotenv
from pydantic import BaseModel, validator, ValidationError


In [ ]:
#| export
class Autor(BaseModel):
    Fname:str 
    Lname:str 
    name:str 
    initials:str
    emails:str
    affiliations:str
    identifier:str 
        
    @validator('name', 'Fname', 'Lname', 'initials', 'affiliations')
    def validate_str(cls, v):
        return v.title()
    
    @validator('identifier', always=True)
    def validate_identifier(cls, v):
        if not v:
            return ''
        else:
            return v

In [ ]:
import nbdev; nbdev.nbdev_export()

In [ ]:
data = {
    'Fname': 'autorFN',
    'Lname': 'autorLN',
    'emails': 'emails',
    'affiliations': 'AFFs', 
    'identifier': 'autorID',
    'name': 'name', 
    'initials': 'autorIN'
       }  #

In [ ]:
Autor.parse_obj(data)

Autor(Fname='Autorfn', Lname='Autorln', name='Name', initials='Autorin', emails='emails', affiliations='Affs', identifier='autorID')

In [ ]:
x = Autor.parse_obj(data)

In [ ]:
x.dict

<bound method BaseModel.dict of Autor(Fname='Autorfn', Lname='Autorln', name='Name', initials='Autorin', emails='emails', affiliations='Affs', identifier='autorID')>

In [ ]:
#| hide
def add_db(self):
    try:
        if self.check_db():
            self.update_db()
        else:
            print('add to database')
            db = TinyDB(self.db_name)
            #updated = date.today()
            #self.updated = updated.strftime('%d-%m-%Y')
            db.insert(self.to_dict())
            db.close()
    except:
        print('Error adding autor to the database')
        return False

def search(self):
    db = TinyDB(self.db_name)#, storage=serialization)
    query = Query()
    result = db.search((query.Lname.matches(self.name)) | (query.Fname.matches(self.name)) | (query.name.matches(self.name)))
    if len(result) > 0:
        db.close()
        return result
    else:
        db.close()
        return False

def check_db(self):
    #if exist return True, else False
    db = TinyDB(self.db_name)#, storage=serialization)
    query = Query()
    if db.search(query.name == self.name):
        db.close()
        return True
    else:
        db.close()
        return False

def update_papers(self):
    db = TinyDB(self.db_name)
    query = Query()
    db.update({'n_papers':self.n_papers},
              query.name == self.name)
    db.close()

def update_db(self):
    db = TinyDB(self.db_name)
    query = Query()
    data = db.get(query.name == self.name)

    autor = self.merge_autors(Autor(data))


#        emails = data['emails']
#        countries = data['countries']
#        ids = data['identifier']
#        # print('obtenidos datos de BD')
#        self.email.extend(emails)
#        self.country.extend(countries)
#        self.id.extend(ids)
#        self.email = list(set(self.email))
#        self.country = list(set(self.country))
#        self.id = list(set(self.id))
    db.update(autor.to_dict() ,
                    query.name == self.name)
    db.close()

def merge_autors(self, autor):
    #merge all variables from 2 autors.
    autor_dict = self.to_dict()
    tmp_dict = autor.to_dict()

    autor_dict['emails'].extend(tmp_dict['emails'])
    autor_dict['countries'].extend(tmp_dict['countries'])
    autor_dict['identifier'].extend(tmp_dict['identifier'])

    autor_dict['n_papers'] = max([autor_dict['n_papers'], tmp_dict['n_papers']])

    autor_dict['affiliations'] = ';'.join([autor_dict['affiliations'], tmp_dict['affiliations']])
    autor_dict['state'] = ';'.join([autor_dict['state'], tmp_dict['state']])

    autor_dict['affiliations'] = ';'.join(list(set(autor_dict['affiliations'].split(';'))))
    autor_dict['emails'] = list(set(autor_dict['emails']))
    autor_dict['countries'] = list(set(autor_dict['countries']))
    autor_dict['identifier'] = list(set(autor_dict['identifier']))

    return Autor(autor_dict)

def to_dict(self):

    parser = {'Fname': self.Fname, 'Lname': self.Lname, 'emails': list(set(self.email)), 'affiliations': self.affiliations,
                    'countries': list(set(self.country)), 'identifier':list(set(self.id)), 'name':self.name,
              'updated':self.updated, 'n_papers':self.n_papers, 'state':self.state}
    return parser